In [ ]:
#!/usr/bin/env python3
"""
MQTT Subscriber für Suspension System
Ersetzt mosquitto_sub für Windows-Entwicklung
"""

import paho.mqtt.client as mqtt
import json
import sys
from datetime import datetime


def on_connect(client, userdata, flags, rc):
	"""Callback für erfolgreiche MQTT-Verbindung"""
	if rc == 0:
		print(f"🔗 Verbunden mit MQTT Broker (localhost:1883)")
		print(f"📡 Abonniere Topic: suspension/#")
		print(f"⏰ {datetime.now().strftime('%H:%M:%S')} - Warte auf Nachrichten...")
		print("-" * 60)

		# Alle suspension Topics abonnieren
		client.subscribe("suspension/#")
	else:
		print(f"❌ Verbindung fehlgeschlagen. Return code: {rc}")


def on_message(client, userdata, msg):
	"""Callback für empfangene MQTT-Nachrichten"""
	timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
	topic = msg.topic

	try:
		# Versuche JSON zu parsen
		payload = json.loads(msg.payload.decode())
		print(f"📨 [{timestamp}] {topic}")
		print(f"    📄 {json.dumps(payload, indent=4, ensure_ascii=False)}")
	except json.JSONDecodeError:
		# Fallback für Nicht-JSON Nachrichten
		payload = msg.payload.decode('utf-8', errors='ignore')
		print(f"📨 [{timestamp}] {topic}")
		print(f"    📄 {payload}")

	print("-" * 60)


def on_disconnect(client, userdata, rc):
	"""Callback für MQTT-Disconnection"""
	print(f"🔌 Verbindung getrennt. Return code: {rc}")


def main():
	"""Hauptfunktion - MQTT Client starten"""
	try:
		# MQTT Client erstellen
		client = mqtt.Client()

		# Callbacks registrieren
		client.on_connect = on_connect
		client.on_message = on_message
		client.on_disconnect = on_disconnect

		# Zu MQTT Broker verbinden
		print("🚀 Starte MQTT Subscriber...")
		print("📍 Broker: localhost:1883")

		client.connect("localhost", 1883, 60)

		# Event-Loop starten
		print("⌨️  Drücken Sie Ctrl+C zum Beenden")
		client.loop_forever()

	except KeyboardInterrupt:
		print("\n🛑 Beendet durch Benutzer")
		client.disconnect()
		sys.exit(0)
	except ConnectionRefusedError:
		print("❌ Fehler: Kann nicht zu MQTT Broker verbinden!")
		print("💡 Stellen Sie sicher, dass ein MQTT Broker läuft:")
		print("   - Docker: docker run -d -p 1883:1883 eclipse-mosquitto")
		print("   - Lokal: mosquitto -v")
		sys.exit(1)
	except Exception as e:
		print(f"❌ Unerwarteter Fehler: {e}")
		sys.exit(1)


if __name__ == "__main__":
	main()